# Chapter 1 Switching,  Lab 2 Configuring Etherchannels
Scripting and adds on created by jpotts15, all other rights reserved by original authors 
github: https://github.com/jpotts15/poor_lazy_ccie_journey
gitpages: https://jpotts15.github.io/poor_lazy_ccie_journey/

This script will help accelerate the labs from "CCIE Enterprise Infrastructure Foundations" by Narbik

Full write up in Chapter 1 Lab 1 and on the blog/ github

## Prework/ Prechecks

0. There are some scripts that talk to the eveng server directly to maybe make things easier, these are safe to skip if your lab is setup, I mainly put these here as a way to reload the base configs as sometimes it can be screwed up

### Skip to here if your lab is running and has IOL L2 images assigned to all the nodes

## Lab work

1. 

'''

# Section 0: Prep work and checking the lab
## Feel free to skip if you are familiar with the labs and don't need to do anything outlined in the overview

## Note: Running the API against EVE will log you out, any labs running will remain running

I made this block basically because my topologies always start with the L3 image when they need the L2, I'll probably also reuse
some of this code later

Overview:

1. We'll get the lab path on the eveng server so that we can interact with the lab programmatically
2. Get the lab, lab device types and ensure they are L2, if not change them by getting available types and changing it
3. anything else i run into later


In [1]:
## Section 0 ##
# for step 0 we'll need the eve-ng sdk (called eve-ng in pypi) written by ttafsir (https://github.com/ttafsir/evengsdk)
# I could write my own api system but thats too far of a tangent and out of scope for this

# Run the command below if in linux/macos
# pip install eve-ng

# if you are using anaconda (recommended):
# On windows open a anaconda prompt (shortcut is created automatically on install) and type: pip install eve-ng
# On Mac if using anaconda easy way to get terminal in your anaconda python install is to go to anaconda navigator, go to enviorments and click the "play" like button next to the enviorment (i.e. base) and click open terminal, then do pip install eve-ng


# if you run this cell it will try running pip for you but really a swing in the dark if it will work
!pip install eve-ng

In [6]:
from evengsdk.client import EvengClient

# IP of your server, no http or anything else
eveng_ip = "192.168.178.35"

# username of your EVE-NG web interface login (default admin)
username = "admin"

# password of your web interface login, (default eve)
password = "eve"

# the filename of the lab, to get this login to eve, go to the folder the lab is in and copy/paste the name out
# default "Chapter-1 Configuring Trunks", don't need file extension but shouldn't matter
lab_filename = "Chapter-2 Configuring etherchannels"
depth = 4

# Part 1, get the path of the lab file, if you actually know this and for whatever reason this part fails you can manually hard code it below
def get_telnet_ports(eveng_ip, username, password, lab):
    telnet_ports = {}
    # Replace this with the actual EVE-NG CLI command to get telnet ports
    client = EvengClient(eveng_ip, log_file="eveng_client_log.log")
    client.login(username=username, password=password)
    #lab = client.api.get_lab(lab)
    lab_nodes_info = client.api.list_nodes(lab)

    for node in lab_nodes_info['data'].values():
        telnet_ports[node['name']] = node['url'][-4:]

    return telnet_ports



def process_folders(client, folder_info, lab_dict, depth):
    if depth == 0:
        return

    folder_list = client.api.get_folder(folder_info['path'])

    for labs_info in folder_list['data']['labs']:
        lab_dict[labs_info['file']] = labs_info['path']

    for sub_folder_info in folder_list['data']['folders']:
        if sub_folder_info['name'] != '..':
            process_folders(client, sub_folder_info, lab_dict, depth - 1)


# Initialize the EvengClient
client = EvengClient(eveng_ip)
client.login(username=username, password=password)

# Initialize an empty dictionary to store lab information
lab_dict = {}

# don't define this variable, if you want to override it do it further down
eveng_lab_filepath = ""

# Get the list of top-level folders
top_level_folder_list = client.api.list_folders()

# Process labs in top-level folders
for labs_info in top_level_folder_list['data']['labs']:
    lab_dict[labs_info['file']] = labs_info['path']

# Check for subfolders in top-level folders
if len(top_level_folder_list['data']['folders']) > 0:
    # Iterate through each subfolder
    for sub_folder_info in top_level_folder_list['data']['folders']:
        if sub_folder_info['name'] != '..':
            process_folders(client, sub_folder_info, lab_dict, depth=2)  # Set the desired depth

for lab_name, path in lab_dict.items():
    if lab_filename in lab_name:
        eveng_lab_filepath = path
        print(f"File Path: {path}")

if len(eveng_lab_filepath) == 0:
    print("Can't find the file specifically; will try being less specific.")
    lab_filename_split = lab_filename.split()

    # Dictionary to store the count of matches for each lab path
    match_counts = {}

    # Check for exact matches
    for lab_name, path in lab_dict.items():
        if lab_filename == lab_name:
            eveng_lab_filepath = path
            print(f"File Path: {path}")
            print("If that is not the right filename, rerun the cell and fill in the line below to manually set it")
            break
        else:
            # Check for partial matches and count the number of matches for each lab path
            matches = sum(word in lab_name for word in lab_filename_split)
            match_counts[path] = matches

    if not eveng_lab_filepath:
        # Find the lab path with the maximum matches
        max_matches_path = max(match_counts, key=match_counts.get)

        # Set the eveng_lab_filepath to the lab path with the most matches
        eveng_lab_filepath = max_matches_path

        print(f"File Path (Partial Match): {eveng_lab_filepath}")
        print("If that is not the right filename, rerun the cell and fill in the line below to manually set it")


# uncomment line below and fill in if you want/know the path
#eveng_lab_filepath = /whatever the path is


if "///" in eveng_lab_filepath:
    eveng_lab_filepath.replace('///','/')
if "//" in eveng_lab_filepath:
    eveng_lab_filepath.replace('//','/')

node_types = client.api.list_node_templates()
print(node_types)
node_details = client.api.list_nodes(eveng_lab_filepath)
print(node_details)

Can't find the file specifically; will try being less specific.
File Path (Partial Match): ///Cisco Press/Switching/Chapter-2 Configuring EtherChannels.unl
If that is not the right filename, rerun the cell and fill in the line below to manually set it
{'code': 200, 'status': 'success', 'message': 'Successfully listed node templates (60003).', 'data': {'a10': 'A10 vThunder.missing', 'alienvault': 'AlienVault Cybersecurity.missing', 'android': 'Android VM.missing', 'macos_simple_kvm': 'Apple MacOS Simple KVM.missing', 'osx': 'Apple OSX.missing', 'aos': 'Apstra AOS Server.missing', 'veos': 'Arista vEOS.missing', 'arubaamp': 'Aruba AirWave.missing', 'clearpass': 'Aruba ClearPass.missing', 'arubanet': 'Aruba NetEdit.missing', 'arubacx': 'Aruba OS-CX Virtual Switch.missing', 'aruba': 'Aruba WiFi Controller.missing', 'barracuda': 'Barraccuda NGIPS.missing', 'brocadevadx': 'Brocade vADX.missing', 'cpsg': 'CheckPoint Security Gateway VE.missing', 'acs': 'Cisco ACS.missing', 'ampcloud': 'Cisco A